In [ ]:
# default_exp model.simsiamae
# default_cls_lvl 3

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# SimSiam Autoencoder

> Simple Siamese Loss for Autoencoder based Representation Learning
> Based on the paper: https://arxiv.org/abs/2011.10566

In [ ]:
# export
import torch
from torch import nn, optim
import torch.nn.functional as F
from deeptool.architecture import Encoder, Decoder, DownUpConv
from deeptool.abs_model import AbsModel
import numpy as np

In [ ]:
# hide

# deal with varying list and batch sizes:

a = list(range(100))
ptr = 99
batch_size = 4
K = 100

ind1 = list(range((ptr + batch_size) % K))
ind2 = list(range(ptr, K))
indexes = ind1 + ind2
indexes
torch.tensor(np.array([1.]))

tensor([1.], dtype=torch.float64)

In [ ]:
# export


class Predictor(nn.Module):
    """
    The Predictor on top of the encoder network
    """

    def __init__(self, in_dim, mid_dim, out_dim, p_drop=0.5):
        """init the classifier"""
        super(Predictor, self).__init__()
        # reduction block
        self.reduce = nn.Sequential(
            nn.Linear(in_dim, mid_dim), nn.Dropout(p=p_drop), nn.ReLU(inplace=True),
        )
        # final block
        self.fin_block = nn.Sequential(nn.Linear(mid_dim, out_dim))

    def forward(self, x):
        """perform forward calculation"""
        # reduce
        x = self.reduce(x)
        x = self.fin_block(x)
        return x

In [ ]:
# export


class SimSiamAE(AbsModel):
    """
    The SimSiam contains the Autoencoder based Architecture and the modified Pretext task
    """

    def __init__(self, device, args):
        """init the network"""
        super(SimSiamAE, self).__init__(args)
        self.device = device  # GPU
        self.dim = args.dim   # 2/3 Dimensional input
        self.n_z = args.n_z   # Compression
        self.ae_mode = args.moco_aemode # inerited from moco

        # Encoder
        self.enc = Encoder(args, vae_mode=False).to(self.device)  # encoder
        
        # Predictor
        self.pred = Predictor(self.n_z, 2 * self.n_z, self.n_z, p_drop=0.2).to(self.device)
        
        # optimizers
        self.optimizerEnc = optim.SGD(self.enc.parameters(), lr=args.lr)
        self.optimizerPred = optim.SGD(self.pred.parameters(), lr=args.lr)
        
        # override prep and take
        self.prep = self.prep_3D if args.dataset_type == "MRNet" else self.prep_2D
        self.take = self.take_3D if args.dataset_type == "MRNet" else self.take_2D
        
        # init the decoder
        self.init_ae(args)
    
    def init_ae(self, args):
        """Init the Autoencoder specific parts"""
        # Decoder
        self.init_dec(args)
        
        # loss function
        self.mse_loss = nn.MSELoss()
    
    def init_dec(self, args):
        """Init a general decoder"""
        self.dec = Decoder(args).to(self.device)  # decoder
        self.optimizerDec = optim.Adam(self.dec.parameters(), lr=args.lr)
    
    def prep_2D(self, data):
        return data[0][0]
    
    def prep_3D(self, data, key="img"):
        return data[key]
    
    def take_2D(self, data):
        return data[0][0], data[0][1]
    
    def take_3D(self, data, key="img"):
        return data[key], data[key]

    def ae_forward(self, x, update):
        """
        Classic regression part of a normal Autoencoder
        """
        x_r = self.dec(z)
        ae_loss = self.mse_loss(x_r, x)
        
        return x_r, ae_loss
    
    def D(self, p, z):
        """
        negative cosine similarity
        """
        # stop gradient on z
        z = z.detach()
        
        # normalize
        p = nn.functional.normalize(p, dim=1)
        z = nn.functional.normalize(z, dim=1)
        
        return -(p * z).sum(dim=-1).mean()

    def forward(self, data, update=True):
        """
        Perform forward computation and update
        """
        # Reset Gradients
        self.optimizerEnc.zero_grad()
        self.optimizerPred.zero_grad()
        self.optimizerDec.zero_grad() if self.ae_mode else None

        # 1. Get the augmented data
        x1, x2 = self.take(data)

        # 2. Send pictures to device
        x1 = x1.to(self.device)
        x2 = x2.to(self.device)
        
        # 3. Encode
        z1 = self.enc(x1)
        z2 = self.enc(x2)
        
        # 4. Predictor
        p1 = self.pred(z1)
        p2 = self.pred(z2)
        
        l_all = 0
        l_d = self.D(p1, z2) / 2 + self.D(p2, z1) / 2
        l_all += l_d
        
        # Add the reconstruction loss
        if self.ae_mode:
            xr = self.dec(z1)
            l_ae = self.mse_loss(xr, x1)
            l_all += l_ae

        # Perform encoder update
        if update:
            l_all.backward()
            
            # Encoder and Precitor
            self.optimizerEnc.step()
            self.optimizerPred.step()
            
            # Decoder
            self.optimizerDec.step() if self.ae_mode else None
            return xr.detach()
        
        else:
            tr_data = {
                "loss_ae": l_ae,
                "loss_D": l_d,
                "l_all": l_all,
            }
            return xr.detach(), tr_data